<a href="https://colab.research.google.com/github/raulpoppiel/sugarcane-yield-prediction/blob/main/03_climatic_data_monthly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><font color="green" size="6"> <b>Sugarcane Yield Prediction using RS and ML<b> </font></center>
<center><img src="https://www.omex.com/wp-content/uploads/2020/10/Sugar-Cane-Harvest-1536x772.jpg" height="150"></center>

---

_Script elaborated by **Raul Roberto Poppiel**_ ([raulpoppiel@gmail.com](raulpoppiel@gmail.com)) for the [FAPESP](https://fapesp.br/en) project Nº [23/01062-1](https://bv.fapesp.br/en/bolsas/207973/satellite-imagery-and-machine-learning-for-sugarcane-yield-estimation-in-regions-of-sao-paulo-state/), led by Professor Ana Claudia dos Santos Luciano (analuciano@usp.br) from ESALQ/USP, Brazil. The methodology employed was developed by Rafaella Pironato Amaro and is detailed in the document titled [Estimativa de produtividade da cana-de-açúcar a partir de imagens do satélite Sentinel-2A e o algoritmo de aprendizagem de máquina Random Forest](https://doi.org/10.11606/D.11.2023.tde-02102023-163947).

### Install and import tools

Geospatial modules

In [ ]:
# Load modules
import ee
import geemap

print('Modules loaded')

Modules loaded


Data science modules

In [ ]:
# Load modules
import os
import pandas as pd
import numpy as np
import time
from pathlib import Path

print('Modules loaded')

Modules loaded


### Connect to GEE and GDrive

In [ ]:
# GEE Authentication
geemap.ee_initialize()

In [ ]:
# Connect to Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Set GDrive paths

Define the output folders in GD

In [ ]:
# Define folder names
folder_name_root = 'Colab Notebooks'
folder_name_project = '01_SugarcaneYieldPrediction'
folder_name_farm = 'usinas_all'
folder_name_specific = '03_climatic_data_monthly'

# define your GEE username
username_gee = 'raulrpoppiel'

# Define sugarcane crop-season (SAFRA)
CropSeason = 1920 # 1920, 2021, 2122, 2223

In [ ]:
# Check if the folder exists or else create
root_path = f'/content/drive/MyDrive/{folder_name_root}'
project_path = f'{root_path}/{folder_name_project}'
farm_path = f'{project_path}/{folder_name_farm}'
out_path = f'{farm_path}/{folder_name_specific}' # your results will be stored in 'out_path'

if not os.path.exists(out_path):
  Path(out_path).mkdir(parents=True, exist_ok=True)
  print("Output directory created successfully.")
else:
  print("Output directory already exists.")

os.chdir(out_path)
print(os.getcwd(),'\n')  # Print the current working directory
pd.DataFrame(os.listdir(), columns=['List files'])  # List files and directories in the current directory

Output directory already exists.
/content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/03_climatic_data_monthly 

List of files:
                                                Files
0  02_climatic_data_monthlySource_MEAN_safra_1920...
1  02_climatic_data_monthlyManual_MEAN_safra_1920...
2  02_climatic_data_monthlyManual_MEAN_safra_2021...
3  02_climatic_data_monthlySource_MEAN_safra_2021...
4  02_climatic_data_monthlyManual_MEAN_safra_2122...
5  02_climatic_data_monthlySource_MEAN_safra_2122...


## Project-specific settings

### Feature collection

Example of attributes table structure:
```
 'ID_SIG': 'X19200890000003600027002', 🟢
 'AREA': 46.0963242244,
 'BLOCO': 2,
 'CODFAZ': 36,
 'COD_USINA': 89,
 'EST_Corte': 1, 🟢
 'Local': 'Usina 3',
 'Ordem': 'Latossolos',
 'POL': 0,
 'SAFRA': '1920', 🟢
 'SAFRA_real': 1819, 🟢
 'TAH': 0,
 'TALHAO': 17002,
 'TCH_ANT': 88.875, 🟢
 'Unidade_So': 'LV21',
 'VAR': 'CTC4',
 'VARIEDADE': 'CTC4',🟢
 'relevo': 'Suave Ondulado',
 'soloGeral': 'LV',
 'usina': 'M3'
```

Import the features from your Assets

In [ ]:
# Import polygons of agricultural plots (shapefiles) from your GEE

fc1 =  ee.FeatureCollection(f'users/{username_gee}/Sugarcane/Safra_{CropSeason}_1')
# fc2 =  ee.FeatureCollection(f'users/{username_gee}/Sugarcane/Safra_{CropSeason}_2')
# fc3 =  ee.FeatureCollection(f'users/{username_gee}/Sugarcane/Safra_{CropSeason}_3')
# fc4 =  ee.FeatureCollection(f'users/{username_gee}/Sugarcane/Safra_{CropSeason}_4')
# fc5 =  ee.FeatureCollection(f'users/{username_gee}/Sugarcane/Safra_{CropSeason}_5')
# fc6 =  ee.FeatureCollection(f'users/{username_gee}/Sugarcane/Safra_{CropSeason}_6')
# fc7 =  ee.FeatureCollection(f'users/{username_gee}/Sugarcane/Safra_{CropSeason}_7')
# fc8 =  ee.FeatureCollection(f'users/{username_gee}/Sugarcane/Safra_{CropSeason}_8')
# fc9 =  ee.FeatureCollection(f'users/{username_gee}/Sugarcane/Safra_{CropSeason}_9')
# fc10 = ee.FeatureCollection(f'users/{username_gee}/Sugarcane/Safra_{CropSeason}_10')
# fc11 = ee.FeatureCollection(f'users/{username_gee}/Sugarcane/Safra_{CropSeason}_11')
# fc12 = ee.FeatureCollection(f'users/{username_gee}/Sugarcane/Safra_{CropSeason}_12')
# fc13 = ee.FeatureCollection(f'users/{username_gee}/Sugarcane/Safra_{CropSeason}_13')
# fc14 = ee.FeatureCollection(f'users/{username_gee}/Sugarcane/Safra_{CropSeason}_14')
# fc15 = ee.FeatureCollection(f'users/{username_gee}/Sugarcane/Safra_{CropSeason}_15')

print('FC1 size: ',fc1.size().getInfo(),'\n')

# Display the table of attributes for one feature/polygon
print('FC1 attribute table (1st feature): \n')
display(fc1.limit(1).getInfo()['features'][0]['properties'])

FC1 size:  321 

FC1 attribute table (1st feature): 



{'AREA': 2.43708017691,
 'BLOCO': '0',
 'CODFAZ': '1',
 'COD_USINA': '28',
 'EST_Corte': '3',
 'ID_SIG': 'X22230280001000100000030',
 'Local': 'Usina 1',
 'Ordem': 'Nitossolos',
 'POL': '0',
 'SAFRA': 2223,
 'SAFRA_real': '2122',
 'TAH': '0',
 'TALHAO': '30',
 'TCH_ANT': '112.79',
 'Unidade_So': 'NV7',
 'VAR': 'CTC4',
 'VARIEDADE': 'CTC4',
 'relevo': 'Ondulado',
 'soloGeral': 'NV',
 'usina': 'M1'}

In [ ]:
# Select an attribute name (column) that contains the ID of polygons
ID = 'ID_SIG'

fc_to_reduction1 = fc1.select(ID)
# fc_to_reduction2 = fc2.select(ID)
# fc_to_reduction3 = fc3.select(ID)
# fc_to_reduction4 = fc4.select(ID)
# fc_to_reduction5 = fc5.select(ID)
# fc_to_reduction6 = fc6.select(ID)
# fc_to_reduction7 = fc7.select(ID)
# fc_to_reduction8 = fc8.select(ID)
# fc_to_reduction9 = fc9.select(ID)
# fc_to_reduction10 = fc10.select(ID)
# fc_to_reduction11 = fc11.select(ID)
# fc_to_reduction12 = fc12.select(ID)
# fc_to_reduction13 = fc13.select(ID)
# fc_to_reduction14 = fc14.select(ID)
# fc_to_reduction15 = fc15.select(ID)

In [ ]:
# Make a colection of collecions of geometries
SugarcaneData = ee.FeatureCollection([fc1
                                      # fc2,fc3,fc4,fc5,fc6,fc7,fc8,fc9,fc10,fc11,fc12,fc13,fc14,fc15
                                      ]).flatten()

# Display the number os polygons
print('FC size: ',SugarcaneData.size().getInfo(),'\n')

# Display the table of attributes for one feature/polygon
print('Attribute table structure (1st feature): \n')
display(SugarcaneData.limit(1).getInfo()['features'][0]['properties'])

FC size:  4815 

Attribute table structure (1st feature): 



{'AREA': 2.43708017691,
 'BLOCO': '0',
 'CODFAZ': '1',
 'COD_USINA': '28',
 'EST_Corte': '3',
 'ID_SIG': 'X22230280001000100000030',
 'Local': 'Usina 1',
 'Ordem': 'Nitossolos',
 'POL': '0',
 'SAFRA': 2223,
 'SAFRA_real': '2122',
 'TAH': '0',
 'TALHAO': '30',
 'TCH_ANT': '112.79',
 'Unidade_So': 'NV7',
 'VAR': 'CTC4',
 'VARIEDADE': 'CTC4',
 'relevo': 'Ondulado',
 'soloGeral': 'NV',
 'usina': 'M1'}

### Crop-season and real period

In [ ]:
# Define the real period in years
if CropSeason == 1920:
    SrtYear = '2018'
    EndYear = '2019'
elif CropSeason == 2021:
    SrtYear = '2019'
    EndYear = '2020'
elif CropSeason == 2122:
    SrtYear = '2020'
    EndYear = '2021'
elif CropSeason == 2223:
    SrtYear = '2021'
    EndYear = '2022'
elif CropSeason == 2324:
    SrtYear = '2022'
    EndYear = '2023'
else:
    # Handle unknown CropSeason values
    print("Unknown CropSeason")

# Example:
# SAFRA_real = SAFRA - 1
SrtYear_real = str(int(SrtYear) - 1)
EndYear_real = str(int(EndYear) - 1)

# Display the results
print(f"Start: 01/04/{SrtYear} --> Real: 01/04/{SrtYear_real}")
print(f"End: 31/03/{EndYear} --> Real: 31/03/{EndYear_real}")

Start: 01/04/2021 --> Real: 01/04/2020
End: 31/03/2022 --> Real: 31/03/2021


In [ ]:
# Use real years for RS data acquisition (DO NOT change it)

# Montly (for using as covariates/predictors)
SrtDate = SrtYear+"-12-01"
EndDate = EndYear+"-03-31"

### Filter Sugarcane field data and bounding box

In [ ]:
# Compute bounding box for the fc
buffer_size = 5000; # define a value in meters

bbox_rect = SugarcaneData.geometry().bounds().buffer(buffer_size).bounds().getInfo()
bbox_coords = bbox_rect.get('coordinates')[0]
bbox = ee.Geometry.Rectangle([bbox_coords[0][0],bbox_coords[0][1],bbox_coords[2][0],bbox_coords[2][1]],None,False)

Map = geemap.Map(basemap='Esri.WorldImagery')
Map.setOptions()

Map.addLayer(SugarcaneData,{'color': 'FF0000'},'fc')
Map.addLayer(bbox, {'color': '#000060'}, 'Bbox')
Map.centerObject(SugarcaneData, 8)

Map.setControlVisibility()
Map

Map(center=[-21.242562902824027, -50.06488035654671], controls=(WidgetControl(options=['position', 'transparen…

## Monthly: manually aggregation

### Import data: T, P, R
Here we get data from the ECMWF ERA-5 land-daily:
* precipitation, in mm = [m * 1000]
* air temperature, in C = [K - 273.15]
* radiation, in MJ/m^2 = [J/m^2 * 0.000001]
* https://developers.google.com/earth-engine/datasets/catalog/ECMWF_ERA5_LAND_DAILY_AGGR

In [ ]:
# Define a function to convert unit and replace the original band
def convert_units(image):
    date = image.get('system:time_start')

    img1 = image.select('temperature_2m').subtract(273.15)
    img2 = image.select('total_precipitation_sum').multiply(1000)
    img3 = image.select('surface_solar_radiation_downwards_sum').multiply(0.000001);
    col = ee.Image.cat(img1,img2,img3)

    return col.set({"system:time_start": ee.Number(date)})

In [ ]:
# Import collection
daily_tp_p_rn = (ee.ImageCollection("ECMWF/ERA5_LAND/DAILY_AGGR")
                  .filterBounds(bbox)
                  .filterDate(SrtDate, EndDate)
                  .map(convert_units)
                  .select(['temperature_2m','total_precipitation_sum','surface_solar_radiation_downwards_sum']))

### Temporal reduction

In [ ]:
# Define the way of aggregation
interval = 1  # time range for aggregation (e.g., every 1 month)
intervalUnit = 'month'  # unit of time (e.g. 'year', 'month', 'day')
temporalReducer_sum = ee.Reducer.sum()  # how to reduce images in time range (sum)
temporalReducer_mean = ee.Reducer.mean()  # how to reduce images in time range (mean)

In [ ]:
# Compute the intervalCount based on the provided start and end dates
intervalCount = ee.Date(EndDate).difference(ee.Date(SrtDate), intervalUnit).divide(interval).ceil().toInt()

# Get time window index sequence.
intervals = ee.List.sequence(0, intervalCount.subtract(1), 1)

# Function for temporal aggregation of images.
def reduceTemporal(i):
    # Calculate temporal composite.
    startRange = ee.Date(SrtDate).advance(ee.Number(interval).multiply(i), intervalUnit)
    endRange = startRange.advance(interval, intervalUnit)
    bandNames = daily_tp_p_rn.first().bandNames().getInfo()

    # Reduce temperature_2m using mean reducer and others using sum reducer
    temporalStat = daily_tp_p_rn.filterDate(startRange, endRange).select('temperature_2m').reduce(temporalReducer_mean).rename(['temperature_2m'])
    sumPrecipitation = daily_tp_p_rn.filterDate(startRange, endRange).select('total_precipitation_sum').reduce(temporalReducer_sum).rename(['total_precipitation_sum'])
    sumRadiation = daily_tp_p_rn.filterDate(startRange, endRange).select('surface_solar_radiation_downwards_sum').reduce(temporalReducer_sum).rename(['surface_solar_radiation_downwards_sum'])

    # Merge all reduced bands into a single image
    composite = temporalStat.addBands([sumPrecipitation, sumRadiation])

    # Set start date as a property.
    composite = composite.set('system:index', ee.Date(startRange).format('YYYYMM'))
    composite = composite.set('system:time_start', ee.Date(startRange).format('YYYY-MM-dd'))

    # Return the composite image.
    return composite

# Map reductions over index sequence to calculate statistics for each interval.
reducedCollection = ee.ImageCollection(intervals.map(reduceTemporal))

In [ ]:
print('The collection size has been reduced to',reducedCollection.size().getInfo(),f'images every {interval} {intervalUnit}')

display('From',reducedCollection.first().get('system:time_start'))
display('To',reducedCollection.sort('system:time_start',opt_ascending=False).first().get('system:time_start'))

The collection size has been reduced to 4 images every 1 month


'From'

'To'

In [ ]:
# Select and display one of the images

# Get the most recent image
clmostRecent = reducedCollection.first().clip(bbox)
print('Month date:',clmostRecent.get('system:index').getInfo(),'\n')

# Get values from image
reducers = ee.Reducer.percentile([1,98], ['min','max']);
stats = clmostRecent.select(['temp.*']).rename('x').reduceRegion(reducer = reducers,
                              geometry = bbox,
                              scale = 5000,
                              tileScale = 16);
img_min = ee.Number(stats.get('x_min').getInfo()).format('%.2f').getInfo();
img_max = ee.Number(stats.get('x_max').getInfo()).format('%.2f').getInfo();
print('\n','PercMin: '+str(img_min)+' (°C)',' |  PercMax: '+str(img_max)+' (°C)','\n')

# Display the image
Map = geemap.Map(basemap='Esri.WorldImagery')
Map.setOptions()

temp_palette = {#'bands': ['temperature_2m_median'],
                 'palette': [ '000080', '0000d9', '4000ff', '8000ff', '0080ff', '00ffff', '00ff80', '80ff00', 'daff00', 'ffff00', 'fff500', 'ffda00', 'ffb000', 'ffa400', 'ff4f00', 'ff2500', 'ff0a00', 'ff00ff', ],
                 'min': [stats.get('x_min')],
                 'max': [stats.get('x_max')]
                 };

Map.addLayer(clmostRecent.select('temp.*'), temp_palette,'Most Recent Image - temperature_2m');
Map.centerObject(bbox, 8);

Map.setControlVisibility()
Map

Month date: 202112 


 PercMin: 22.53 (°C)  |  PercMax: 26.54 (°C) 



Map(center=[-21.398819841588608, -49.33270043259046], controls=(WidgetControl(options=['position', 'transparen…

In [ ]:
# Define the location of interest with a point.
lat = -21.2126
lon =  -50.8942
poi = ee.Geometry.Point(lon, lat)

# Explore local values.
local_cl = reducedCollection.getRegion(poi, 11132).getInfo()

local_cl = pd.DataFrame(local_cl[1:], columns= pd.DataFrame(local_cl).iloc[0])
local_cl

,id,longitude,latitude,time,temperature_2m,total_precipitation_sum,surface_solar_radiation_downwards_sum
0,202112,-50.850233,-21.250097,2021-12-01,26.187631,100.000168,774.862012
1,202201,-50.850233,-21.250097,2022-01-01,26.363640,202.172495,653.222024
2,202202,-50.850233,-21.250097,2022-02-01,26.170045,112.984827,637.786824
3,202203,-50.850233,-21.250097,2022-03-01,26.587633,103.470127,594.930748


### Reduce by region and export as tables

In [ ]:
# Define the statistic
# "MEAN", "MAXIMUM", "MEDIAN","MINIMUM","MODE","STD","MIN_MAX","SUM","VARIANCE", "COUNT"
statistics_Type = 'MEAN'

# Define the output path to save results
global_stats_path1 =  os.path.join(out_path, f'02_climatic_data_monthlyManual_{statistics_Type}_safra_{CropSeason}_1.csv')
# global_stats_path2 =  os.path.join(out_path, f'02_climatic_data_monthlyManual_{statistics_Type}_safra_{CropSeason}_2.csv')
# global_stats_path3 =  os.path.join(out_path, f'02_climatic_data_monthlyManual_{statistics_Type}_safra_{CropSeason}_3.csv')
# global_stats_path4 =  os.path.join(out_path, f'02_climatic_data_monthlyManual_{statistics_Type}_safra_{CropSeason}_4.csv')
# global_stats_path5 =  os.path.join(out_path, f'02_climatic_data_monthlyManual_{statistics_Type}_safra_{CropSeason}_5.csv')
# global_stats_path6 =  os.path.join(out_path, f'02_climatic_data_monthlyManual_{statistics_Type}_safra_{CropSeason}_6.csv')
# global_stats_path7 =  os.path.join(out_path, f'02_climatic_data_monthlyManual_{statistics_Type}_safra_{CropSeason}_7.csv')
# global_stats_path8 =  os.path.join(out_path, f'02_climatic_data_monthlyManual_{statistics_Type}_safra_{CropSeason}_8.csv')
# global_stats_path9 =  os.path.join(out_path, f'02_climatic_data_monthlyManual_{statistics_Type}_safra_{CropSeason}_9.csv')
# global_stats_path10 = os.path.join(out_path, f'02_climatic_data_monthlyManual_{statistics_Type}_safra_{CropSeason}_10.csv')
# global_stats_path11 = os.path.join(out_path, f'02_climatic_data_monthlyManual_{statistics_Type}_safra_{CropSeason}_11.csv')
# global_stats_path12 = os.path.join(out_path, f'02_climatic_data_monthlyManual_{statistics_Type}_safra_{CropSeason}_12.csv')
# global_stats_path13 = os.path.join(out_path, f'02_climatic_data_monthlyManual_{statistics_Type}_safra_{CropSeason}_13.csv')
# global_stats_path14 = os.path.join(out_path, f'02_climatic_data_monthlyManual_{statistics_Type}_safra_{CropSeason}_14.csv')
# global_stats_path15 = os.path.join(out_path, f'02_climatic_data_monthlyManual_{statistics_Type}_safra_{CropSeason}_15.csv')

print(global_stats_path1)
# print(global_stats_path2)
# print(global_stats_path3)
# print(global_stats_path4)
# print(global_stats_path5)
# print(global_stats_path6)
# print(global_stats_path7)
# print(global_stats_path8)
# print(global_stats_path9)
# print(global_stats_path10)
# print(global_stats_path11)
# print(global_stats_path12)
# print(global_stats_path13)
# print(global_stats_path14)
# print(global_stats_path15)

/content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/03_climatic_data_monthly/02_climatic_data_monthlyManual_MEAN_safra_2223_1.csv
/content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/03_climatic_data_monthly/02_climatic_data_monthlyManual_MEAN_safra_2223_2.csv
/content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/03_climatic_data_monthly/02_climatic_data_monthlyManual_MEAN_safra_2223_3.csv
/content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/03_climatic_data_monthly/02_climatic_data_monthlyManual_MEAN_safra_2223_4.csv
/content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/03_climatic_data_monthly/02_climatic_data_monthlyManual_MEAN_safra_2223_5.csv
/content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/03_climatic_data_monthly/02_climatic_data_monthlyManual_MEAN_safra_2223_6.csv
/content/drive/MyDrive/Colab Notebooks/01_Suga

In [ ]:
# Reduzing daily climatic values by zone
scale = 100.0

parameters = {
    'in_value_raster': reducedCollection,
    'statistics_type':statistics_Type,
    'scale':float(scale), # a high scale avoid missing values
    'tile_scale':16.0,
    'return_fc':False,
    'timeout':3000
    }

In [ ]:
geemap.zonal_statistics(in_zone_vector=fc_to_reduction1,out_file_path=global_stats_path1,**parameters)
time.sleep(2)
# geemap.zonal_statistics(in_zone_vector=fc_to_reduction2,out_file_path=global_stats_path2,**parameters)
# time.sleep(2)
# geemap.zonal_statistics(in_zone_vector=fc_to_reduction3,out_file_path=global_stats_path3,**parameters)
# time.sleep(2)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/03_climatic_data_monthly/02_climatic_data_monthlyManual_MEAN_safra_2223_1.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/03_climatic_data_monthly/02_climatic_data_monthlyManual_MEAN_safra_2223_2.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/03_climatic_data_monthly/02_climatic_data_monthlyManual_MEAN_safra_2223_3.csv


In [ ]:
# geemap.zonal_statistics(in_zone_vector=fc_to_reduction4,out_file_path=global_stats_path4,**parameters)
# time.sleep(2)
# geemap.zonal_statistics(in_zone_vector=fc_to_reduction5,out_file_path=global_stats_path5,**parameters)
# time.sleep(2)
# geemap.zonal_statistics(in_zone_vector=fc_to_reduction6,out_file_path=global_stats_path6,**parameters)
# time.sleep(2)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/03_climatic_data_monthly/02_climatic_data_monthlyManual_MEAN_safra_2223_4.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/03_climatic_data_monthly/02_climatic_data_monthlyManual_MEAN_safra_2223_5.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/03_climatic_data_monthly/02_climatic_data_monthlyManual_MEAN_safra_2223_6.csv


In [ ]:
# geemap.zonal_statistics(in_zone_vector=fc_to_reduction7,out_file_path=global_stats_path7,**parameters)
# time.sleep(2)
# geemap.zonal_statistics(in_zone_vector=fc_to_reduction8,out_file_path=global_stats_path8,**parameters)
# time.sleep(2)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/03_climatic_data_monthly/02_climatic_data_monthlyManual_MEAN_safra_2223_7.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/03_climatic_data_monthly/02_climatic_data_monthlyManual_MEAN_safra_2223_8.csv


In [ ]:
# geemap.zonal_statistics(in_zone_vector=fc_to_reduction9,out_file_path=global_stats_path9,**parameters)
# time.sleep(2)
# geemap.zonal_statistics(in_zone_vector=fc_to_reduction10,out_file_path=global_stats_path10,**parameters)
# time.sleep(2)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/03_climatic_data_monthly/02_climatic_data_monthlyManual_MEAN_safra_2223_9.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/03_climatic_data_monthly/02_climatic_data_monthlyManual_MEAN_safra_2223_10.csv


In [ ]:
# geemap.zonal_statistics(in_zone_vector=fc_to_reduction11,out_file_path=global_stats_path11,**parameters)
# time.sleep(2)
# geemap.zonal_statistics(in_zone_vector=fc_to_reduction12,out_file_path=global_stats_path12,**parameters)
# time.sleep(2)
# geemap.zonal_statistics(in_zone_vector=fc_to_reduction13,out_file_path=global_stats_path13,**parameters)
# time.sleep(2)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/03_climatic_data_monthly/02_climatic_data_monthlyManual_MEAN_safra_2223_11.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/03_climatic_data_monthly/02_climatic_data_monthlyManual_MEAN_safra_2223_12.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/03_climatic_data_monthly/02_climatic_data_monthlyManual_MEAN_safra_2223_13.csv


In [ ]:
# geemap.zonal_statistics(in_zone_vector=fc_to_reduction14,out_file_path=global_stats_path14,**parameters)
# time.sleep(2)
# geemap.zonal_statistics(in_zone_vector=fc_to_reduction15,out_file_path=global_stats_path15,**parameters)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/03_climatic_data_monthly/02_climatic_data_monthlyManual_MEAN_safra_2223_14.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/03_climatic_data_monthly/02_climatic_data_monthlyManual_MEAN_safra_2223_15.csv


### Merge tables into a single one

In [ ]:
# Define the paths to your CSV files
file_paths = [
    os.path.join(out_path, f'02_climatic_data_monthlyManual_{statistics_Type}_safra_{CropSeason}_{i}.csv')
    for i in range(1, 16)
]

print(len(file_paths), 'paths')

15 paths


In [ ]:
# Create an empty DataFrame to store the merged data
merged_df = pd.DataFrame()

# Loop through the CSV files and merge them
for file_path in file_paths:
    if os.path.exists(file_path):
        # Read each CSV file into a DataFrame
        df = pd.read_csv(file_path)

        # Remove the "system:index" column if it exists
        if "system:index" in df.columns:
            df = df.drop(columns=["system:index"])

        # Merge the data into the merged_df
        if merged_df.empty:
            merged_df = df
        else:  # Fix the typo here (remove "ID")
            merged_df = pd.concat([merged_df, df], axis=0, ignore_index=True)

# Move the last column (ID_SIG) to the first position
merged_df = merged_df[[merged_df.columns[-1]] + list(merged_df.columns[:-1])]
print('Dataframes merged!')

Dataframes merged!


In [ ]:
# Define the path for the output merged CSV file
merged_csv_path = os.path.join(out_path, f'02_climatic_data_monthlyManual_{statistics_Type}_safra_{CropSeason}_merged.csv')

# Save the merged DataFrame to a CSV file
merged_df.to_csv(merged_csv_path, encoding='utf-8', index=False)

# Print the path to the merged CSV file
print(f'Merged CSV file saved at: {merged_csv_path}')

Merged CSV file saved at: /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/03_climatic_data_monthly/02_climatic_data_monthlyManual_MEAN_safra_2223_merged.csv


In [ ]:
# Delete CSV files
for file_path in file_paths:
    try:
        os.remove(file_path)
        print(f"Deleted: {file_path}")
    except FileNotFoundError:
        print(f"File not found: {file_path}")
    except Exception as e:
        print(f"Error deleting {file_path}: {e}")

Deleted: /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/03_climatic_data_monthly/02_climatic_data_monthlyManual_MEAN_safra_2223_1.csv
Deleted: /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/03_climatic_data_monthly/02_climatic_data_monthlyManual_MEAN_safra_2223_2.csv
Deleted: /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/03_climatic_data_monthly/02_climatic_data_monthlyManual_MEAN_safra_2223_3.csv
Deleted: /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/03_climatic_data_monthly/02_climatic_data_monthlyManual_MEAN_safra_2223_4.csv
Deleted: /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/03_climatic_data_monthly/02_climatic_data_monthlyManual_MEAN_safra_2223_5.csv
Deleted: /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/03_climatic_data_monthly/02_climatic_data_monthlyManual_MEAN_safra_222

.

.

## Monthly: aggregated at source

### Import data: T, P, R
Here we get data from the ECMWF ERA-5 land-monthly:
* total precipitation, in mm = [m * 1000]
* mean air temperature, in °C = [K - 273.15]
* radiation, in MJ/m^2 = [J/m^2 * 0.000001]
* https://developers.google.com/earth-engine/datasets/catalog/ECMWF_ERA5_LAND_MONTHLY_AGGR

In [ ]:
# Define a function to convert unit and replace the original band
def convert_units(image):
    date = image.get('system:time_start')

    img1 = image.select('temperature_2m').subtract(273.15)
    img2 = image.select('total_precipitation_sum').multiply(1000)
    img3 = image.select('surface_solar_radiation_downwards_sum').multiply(0.000001);
    col = ee.Image.cat(img1,img2,img3)

    return col.set({"system:time_start": ee.Number(date)})

In [ ]:
# Import collection
monthly_tp_p_rn = (ee.ImageCollection("ECMWF/ERA5_LAND/MONTHLY_AGGR")
                  .filterBounds(bbox)
                  .filterDate(SrtDate, EndDate)
                  .map(convert_units)
                  .select(['temperature_2m','total_precipitation_sum','surface_solar_radiation_downwards_sum']))

In [ ]:
# Get the most recent image for visualization
clmostRecent1 = monthly_tp_p_rn.first().clip(bbox)
print('Month date:',clmostRecent1.get('system:index').getInfo(),'\n')

# Get values from the image
reducers = ee.Reducer.percentile([1,98], ['min','max']);
stats = clmostRecent1.select(['temperature_2m']).rename('x').reduceRegion(reducer = reducers,
                              geometry = bbox,
                              scale = 5000,
                              tileScale = 16);
img_min = ee.Number(stats.get('x_min').getInfo()).format('%.2f').getInfo();
img_max = ee.Number(stats.get('x_max').getInfo()).format('%.2f').getInfo();
print('\n','PercMin: '+str(img_min)+' (°C)',' |  PercMax: '+str(img_max)+' (°C)','\n')

# Display the image
Map = geemap.Map(basemap='Esri.WorldImagery')
Map.setOptions()

temp_palette = {'bands': ['temperature_2m'],
                 'palette': [ '000080', '0000d9', '4000ff', '8000ff', '0080ff', '00ffff', '00ff80', '80ff00', 'daff00', 'ffff00', 'fff500', 'ffda00', 'ffb000', 'ffa400', 'ff4f00', 'ff2500', 'ff0a00', 'ff00ff', ],
                 'min': [stats.get('x_min')],
                 'max': [stats.get('x_max')]
                 };

Map.addLayer(clmostRecent1, temp_palette,'Most Recent Image - temperature_2m');
Map.centerObject(bbox, 8);

Map.setControlVisibility()
Map

Month date: 202112 


 PercMin: 22.64 (°C)  |  PercMax: 26.55 (°C) 



Map(center=[-21.398819841588608, -49.33270043259046], controls=(WidgetControl(options=['position', 'transparen…

In [ ]:
# Define the location of interest with a point.
lat = -21.2126
lon =  -50.8942
poi = ee.Geometry.Point(lon, lat)

# Explore local values.
local_cl = monthly_tp_p_rn.getRegion(poi, 11132).getInfo()

local_cl = pd.DataFrame(local_cl[1:], columns= pd.DataFrame(local_cl).iloc[0])
local_cl["datetime"] = pd.to_datetime(local_cl["time"], unit="ms")
local_cl = local_cl.set_index("datetime")
local_cl

,id,longitude,latitude,time,temperature_2m,total_precipitation_sum,surface_solar_radiation_downwards_sum
datetime,,,,,,,
2021-12-01,202112,-50.850233,-21.250097,1638316800000,26.187631,100.000168,774.862012
2022-01-01,202201,-50.850233,-21.250097,1640995200000,26.363640,202.172495,653.222024
2022-02-01,202202,-50.850233,-21.250097,1643673600000,26.170045,112.984827,637.786824
2022-03-01,202203,-50.850233,-21.250097,1646092800000,26.549194,110.776107,611.863804


### Reduce by region and export as tables

In [ ]:
# Define the statistic
# "MEAN", "MAXIMUM", "MEDIAN","MINIMUM","MODE","STD","MIN_MAX","SUM","VARIANCE", "COUNT"
statistics_Type = 'MEAN'

# Define the output path to save results
global_stats_path1 =  os.path.join(out_path, f'02_climatic_data_monthlySource_{statistics_Type}_safra_{CropSeason}_1.csv')
# global_stats_path2 =  os.path.join(out_path, f'02_climatic_data_monthlySource_{statistics_Type}_safra_{CropSeason}_2.csv')
# global_stats_path3 =  os.path.join(out_path, f'02_climatic_data_monthlySource_{statistics_Type}_safra_{CropSeason}_3.csv')
# global_stats_path4 =  os.path.join(out_path, f'02_climatic_data_monthlySource_{statistics_Type}_safra_{CropSeason}_4.csv')
# global_stats_path5 =  os.path.join(out_path, f'02_climatic_data_monthlySource_{statistics_Type}_safra_{CropSeason}_5.csv')
# global_stats_path6 =  os.path.join(out_path, f'02_climatic_data_monthlySource_{statistics_Type}_safra_{CropSeason}_6.csv')
# global_stats_path7 =  os.path.join(out_path, f'02_climatic_data_monthlySource_{statistics_Type}_safra_{CropSeason}_7.csv')
# global_stats_path8 =  os.path.join(out_path, f'02_climatic_data_monthlySource_{statistics_Type}_safra_{CropSeason}_8.csv')
# global_stats_path9 =  os.path.join(out_path, f'02_climatic_data_monthlySource_{statistics_Type}_safra_{CropSeason}_9.csv')
# global_stats_path10 = os.path.join(out_path, f'02_climatic_data_monthlySource_{statistics_Type}_safra_{CropSeason}_10.csv')
# global_stats_path11 = os.path.join(out_path, f'02_climatic_data_monthlySource_{statistics_Type}_safra_{CropSeason}_11.csv')
# global_stats_path12 = os.path.join(out_path, f'02_climatic_data_monthlySource_{statistics_Type}_safra_{CropSeason}_12.csv')
# global_stats_path13 = os.path.join(out_path, f'02_climatic_data_monthlySource_{statistics_Type}_safra_{CropSeason}_13.csv')
# global_stats_path14 = os.path.join(out_path, f'02_climatic_data_monthlySource_{statistics_Type}_safra_{CropSeason}_14.csv')
# global_stats_path15 = os.path.join(out_path, f'02_climatic_data_monthlySource_{statistics_Type}_safra_{CropSeason}_15.csv')

print(global_stats_path1)
# print(global_stats_path2)
# print(global_stats_path3)
# print(global_stats_path4)
# print(global_stats_path5)
# print(global_stats_path6)
# print(global_stats_path7)
# print(global_stats_path8)
# print(global_stats_path9)
# print(global_stats_path10)
# print(global_stats_path11)
# print(global_stats_path12)
# print(global_stats_path13)
# print(global_stats_path14)
# print(global_stats_path15)

/content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/03_climatic_data_monthly/02_climatic_data_monthlySource_MEAN_safra_2223_1.csv
/content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/03_climatic_data_monthly/02_climatic_data_monthlySource_MEAN_safra_2223_2.csv
/content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/03_climatic_data_monthly/02_climatic_data_monthlySource_MEAN_safra_2223_3.csv
/content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/03_climatic_data_monthly/02_climatic_data_monthlySource_MEAN_safra_2223_4.csv
/content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/03_climatic_data_monthly/02_climatic_data_monthlySource_MEAN_safra_2223_5.csv
/content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/03_climatic_data_monthly/02_climatic_data_monthlySource_MEAN_safra_2223_6.csv
/content/drive/MyDrive/Colab Notebooks/01_Suga

In [ ]:
# Reduzing daily climatic values by zone
scale = 100.0

parameters = {
    'in_value_raster': monthly_tp_p_rn,
    'statistics_type':statistics_Type,
    'scale':float(scale), # a high scale avoid missing values
    'tile_scale':16.0,
    'return_fc':False,
    'timeout':3000
    }

In [ ]:
geemap.zonal_statistics(in_zone_vector=fc_to_reduction1,out_file_path=global_stats_path1,**parameters)
time.sleep(2)
# geemap.zonal_statistics(in_zone_vector=fc_to_reduction2,out_file_path=global_stats_path2,**parameters)
# time.sleep(2)
# geemap.zonal_statistics(in_zone_vector=fc_to_reduction3,out_file_path=global_stats_path3,**parameters)
# time.sleep(2)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/03_climatic_data_monthly/02_climatic_data_monthlySource_MEAN_safra_2223_1.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/03_climatic_data_monthly/02_climatic_data_monthlySource_MEAN_safra_2223_2.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/03_climatic_data_monthly/02_climatic_data_monthlySource_MEAN_safra_2223_3.csv


In [ ]:
# geemap.zonal_statistics(in_zone_vector=fc_to_reduction4,out_file_path=global_stats_path4,**parameters)
# time.sleep(2)
# geemap.zonal_statistics(in_zone_vector=fc_to_reduction5,out_file_path=global_stats_path5,**parameters)
# time.sleep(2)
# geemap.zonal_statistics(in_zone_vector=fc_to_reduction6,out_file_path=global_stats_path6,**parameters)
# time.sleep(2)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/03_climatic_data_monthly/02_climatic_data_monthlySource_MEAN_safra_2223_4.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/03_climatic_data_monthly/02_climatic_data_monthlySource_MEAN_safra_2223_5.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/03_climatic_data_monthly/02_climatic_data_monthlySource_MEAN_safra_2223_6.csv


In [ ]:
# geemap.zonal_statistics(in_zone_vector=fc_to_reduction7,out_file_path=global_stats_path7,**parameters)
# time.sleep(2)
# geemap.zonal_statistics(in_zone_vector=fc_to_reduction8,out_file_path=global_stats_path8,**parameters)
# time.sleep(2)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/03_climatic_data_monthly/02_climatic_data_monthlySource_MEAN_safra_2223_7.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/03_climatic_data_monthly/02_climatic_data_monthlySource_MEAN_safra_2223_8.csv


In [ ]:
# geemap.zonal_statistics(in_zone_vector=fc_to_reduction9,out_file_path=global_stats_path9,**parameters)
# time.sleep(2)
# geemap.zonal_statistics(in_zone_vector=fc_to_reduction10,out_file_path=global_stats_path10,**parameters)
# time.sleep(2)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/03_climatic_data_monthly/02_climatic_data_monthlySource_MEAN_safra_2223_9.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/03_climatic_data_monthly/02_climatic_data_monthlySource_MEAN_safra_2223_10.csv


In [ ]:
# geemap.zonal_statistics(in_zone_vector=fc_to_reduction11,out_file_path=global_stats_path11,**parameters)
# time.sleep(2)
# geemap.zonal_statistics(in_zone_vector=fc_to_reduction12,out_file_path=global_stats_path12,**parameters)
# time.sleep(2)
# geemap.zonal_statistics(in_zone_vector=fc_to_reduction13,out_file_path=global_stats_path13,**parameters)
# time.sleep(2)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/03_climatic_data_monthly/02_climatic_data_monthlySource_MEAN_safra_2223_11.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/03_climatic_data_monthly/02_climatic_data_monthlySource_MEAN_safra_2223_12.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/03_climatic_data_monthly/02_climatic_data_monthlySource_MEAN_safra_2223_13.csv


In [ ]:
# geemap.zonal_statistics(in_zone_vector=fc_to_reduction14,out_file_path=global_stats_path14,**parameters)
# time.sleep(2)
# geemap.zonal_statistics(in_zone_vector=fc_to_reduction15,out_file_path=global_stats_path15,**parameters)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/03_climatic_data_monthly/02_climatic_data_monthlySource_MEAN_safra_2223_14.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/03_climatic_data_monthly/02_climatic_data_monthlySource_MEAN_safra_2223_15.csv


### Merge tables into a single one

In [ ]:
# Define the paths to your CSV files
file_paths = [
    os.path.join(out_path, f'02_climatic_data_monthlySource_{statistics_Type}_safra_{CropSeason}_{i}.csv')
    for i in range(1, 16)
]

print(len(file_paths), 'paths')

15 paths


In [ ]:
# Create an empty DataFrame to store the merged data
merged_df = pd.DataFrame()

# Loop through the CSV files and merge them
for file_path in file_paths:
    if os.path.exists(file_path):
        # Read each CSV file into a DataFrame
        df = pd.read_csv(file_path)

        # Remove the "system:index" column if it exists
        if "system:index" in df.columns:
            df = df.drop(columns=["system:index"])

        # Merge the data into the merged_df
        if merged_df.empty:
            merged_df = df
        else:  # Fix the typo here (remove "ID")
            merged_df = pd.concat([merged_df, df], axis=0, ignore_index=True)

# Move the last column (ID_SIG) to the first position
merged_df = merged_df[[merged_df.columns[-1]] + list(merged_df.columns[:-1])]
print('Dataframes merged!')

Dataframes merged!


In [ ]:
# Define the path for the output merged CSV file
merged_csv_path = os.path.join(out_path, f'02_climatic_data_monthlySource_{statistics_Type}_safra_{CropSeason}_merged.csv')

# Save the merged DataFrame to a CSV file
merged_df.to_csv(merged_csv_path, encoding='utf-8', index=False)

# Print the path to the merged CSV file
print(f'Merged CSV file saved at: {merged_csv_path}')

Merged CSV file saved at: /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/03_climatic_data_monthly/02_climatic_data_monthlySource_MEAN_safra_2223_merged.csv


In [ ]:
# Delete CSV files
for file_path in file_paths:
    try:
        os.remove(file_path)
        print(f"Deleted: {file_path}")
    except FileNotFoundError:
        print(f"File not found: {file_path}")
    except Exception as e:
        print(f"Error deleting {file_path}: {e}")

Deleted: /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/03_climatic_data_monthly/02_climatic_data_monthlySource_MEAN_safra_2223_1.csv
Deleted: /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/03_climatic_data_monthly/02_climatic_data_monthlySource_MEAN_safra_2223_2.csv
Deleted: /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/03_climatic_data_monthly/02_climatic_data_monthlySource_MEAN_safra_2223_3.csv
Deleted: /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/03_climatic_data_monthly/02_climatic_data_monthlySource_MEAN_safra_2223_4.csv
Deleted: /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/03_climatic_data_monthly/02_climatic_data_monthlySource_MEAN_safra_2223_5.csv
Deleted: /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/03_climatic_data_monthly/02_climatic_data_monthlySource_MEAN_safra_222